<a href="https://colab.research.google.com/github/kk98033/NTTU-Digital-System-Design-Lab-Project/blob/main/%E9%80%8F%E9%81%8E_RAG_%E6%8A%80%E8%A1%93%E5%BC%B7%E5%8C%96_LLM_%E8%B3%87%E8%A8%8A%E6%AA%A2%E7%B4%A2%E4%B8%A6%E6%98%8E%E7%A2%BA%E5%91%88%E7%8F%BE%E4%BE%86%E6%BA%90%E8%BF%BD%E6%BA%AF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install milvus
!pip install milvus python-dotenv
!pip install llama-index
!pip install llama-index-vector-stores-milvus

In [ ]:
# 卸載 grpcio，然後安裝特定版本的 grpcio（1.60.0）安裝其他的版本可能會導致衝突
!pip uninstall grpcio
!pip install grpcio==1.60.0

Found existing installation: grpcio 1.60.0
Uninstalling grpcio-1.60.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/grpc/*
    /usr/local/lib/python3.10/dist-packages/grpcio-1.60.0.dist-info/*
Proceed (Y/n)? y
  Successfully uninstalled grpcio-1.60.0
  Using cached grpcio-1.60.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.4 MB)


In [ ]:
import os
from google.colab import userdata

# 設定OpenAI API密鑰
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.core.query_engine import CitationQueryEngine
from llama_index.core.indices.vector_store.base import VectorStoreIndex
from llama_index.core import SimpleDirectoryReader
from llama_index.core import StorageContext
# from llama_index.vector_stores.milvus import MilvusVectorStore
from milvus import default_server

from llama_index.core import Settings
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter

In [ ]:
from llama_index.vector_stores.milvus import MilvusVectorStore

In [ ]:
!wget 'https://drive.google.com/uc?export=download&id=1kDwfFbMC3nM0K7OTXu88cczYBFm10pQT' -O 原住民資料.pdf

--2024-04-03 05:56:22--  https://drive.google.com/uc?export=download&id=1kDwfFbMC3nM0K7OTXu88cczYBFm10pQT
Resolving drive.google.com (drive.google.com)... 74.125.128.138, 74.125.128.113, 74.125.128.100, ...
Connecting to drive.google.com (drive.google.com)|74.125.128.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1kDwfFbMC3nM0K7OTXu88cczYBFm10pQT&export=download [following]
--2024-04-03 05:56:22--  https://drive.usercontent.google.com/download?id=1kDwfFbMC3nM0K7OTXu88cczYBFm10pQT&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.250.145.132, 2a00:1450:4013:c14::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.250.145.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5073207 (4.8M) [application/octet-stream]
Saving to: ‘原住民資料.pdf’

原住民資料.pdf      100%[===================>]   4.84M  26.3MB/s    in 0

In [ ]:
# 從 PDF 文件中讀取文檔數據。
documents = SimpleDirectoryReader(
    input_files=["./原住民資料.pdf"]
  ).load_data()
print(len(documents))
# print(documents)

In [ ]:
# 啟動 Milvus lite 服務。
default_server.start()
vector_dimension = 768

vector_store = MilvusVectorStore(
    collection_name="citations",
    host="127.0.0.1",
    port=default_server.listen_port,
    dim=vector_dimension
)

DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: bbe1e464a2c4429da83f7784aed9d22c
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created collection: citations
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created an index on collection: citations


In [ ]:
# 從文檔中創建向量存儲索引。
index = VectorStoreIndex.from_documents(documents)

In [ ]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

# 設置 LLaMA 模型、嵌入模型、節點解析器等。
Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")
Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
Settings.num_output = 512
Settings.context_window = 3900

In [ ]:
# 初始化引擎，設置相似性搜索的參數。
query_engine = CitationQueryEngine.from_args(
    index,
    similarity_top_k=3,
    citation_chunk_size=512,
)
# response = query_engine.query("魯凱一詞的來源是什麼")
response = query_engine.query("阿美族有什麼祭典，請詳細講解")
print(response)
print('=================')
for source in response.source_nodes:
    print(source.node.get_text())

阿美族有豐年祭和漁撈祭兩個主要的祭典活動。豐年祭是在小米或稻米收穫後舉行的祭典，用來感謝神靈並祈求豐收、健康、以及族人平安。這個祭典包含了豐收、謝神、聯誼、社交、年齡階級晉級儀式、軍事訓練驗收儀式等多種活動，具有經濟、宗教、社會、政治、文化等多性質的綜合活動[5]。漁撈祭則是在每年5、6月間舉行的祭祀海神或河川神的活動，用來祈求出海平安或撈捕魚類滿載而歸。漁撈祭包含海祭和河祭兩種形式，族人會在海邊或河川進行相應的祭祀活動，並將捕獲的魚類烹煮後按照年齡階級的輩份順序送交長者、耆老品嚐，以表示敬老尊賢的意義[6]。這兩個祭典活動展現了阿美族豐富的宗教信仰和文化傳統。
Source 1:
母系社會承繼制度正顯示阿美族在家族事務與部落公共事務上，男女有別、社會分工的文化特色。 風俗民情 阿美族的傳統信仰相信天地萬物都有靈性，以kawas（靈魂）為核心概念，有神、鬼、人、動物、植物等不同類別；kawas 也因為屬性不同而在空間上有天上、地下靈不同層次的分類；天上神有天神、太陽神、月亮神等，地下靈有河神、海神、土地神、動物神等，可以說是一個秉持萬物有靈、多神信仰觀念的民族。阿美族傳統信仰中的巫師，稱為cikawasay 或sikawasay，是專職的宗教人員，用占卜的神靈溝通儀式為族人消災、治病。巫師除了幫個別族人招吉避兇外，也會在部落農耕儀式與狩獵前後的各項儀式中，向天神祈福或感謝。

Source 2:
戰後中華民國政府時期，基督宗教傳入阿美族部落，以長老教會、天主教、聖教會、真耶穌教會等信徒較多，基督教的牧師或是天主教的神父，在部落中取代了巫師許多的祈福儀式與服務工作，成為目前阿美族部落中最普遍、主要的宗教專職人員。目前阿美族部落中除了傳統宗教、西方宗教外，在臺東、恆春半島也有信仰漢族民間宗教信仰的部落，整體呈現多元並存的宗教現象。  1. 豐年祭（ilisin/malaikid/malikoda/kiluma’an/zukimisai/siukakusai） 豐年祭是阿美族人在小米收穫後，族人為感謝神靈而舉行的祭典。豐年祭的名稱，各部落因說法不一，有malalikid、malikoda、kiluma’an 等說法，後來受日本語影響而有zukimisai（月見祭）與siukakukai（收穫會）等不同說法，現在多稱為豐年祭。豐年祭原是小米收穫後進行，在部落稻米種植面積大量

In [ ]:
response = query_engine.query("台灣最高的樓是什麼")
print(response)
print('=================')
for source in response.source_nodes:
    print(source.node.get_text())

Unfortunately, none of the provided sources contain information about the tallest building in Taiwan.
Source 1:
● minfazfaz 迎最高祖靈至祖靈屋（8 月 11 日）。 ● manqatubi 在部落中「踩街趕煞」繞境（8 月 12 日至 21 日）。 ● 最高祖靈巡各邵族家屋（8 月 22 日至 23 日）。 ● 拆祖靈屋（8 月 28 日）。         最精彩的邵族大過年儀式為「minrikus（最後祭）」儀式，熱鬧非凡、通宵達旦，每戶人家鞭炮、歌聲不絕於耳，部落中之漢族和觀光客非常踴躍參與。        當部落裡每戶族人家都完成最高祖靈祈福後，已經是隔天清晨接近中午時刻。隔數日，女祭司祭拜完祖靈屋，送走祖靈後，會將祖靈屋拆除、火燒，代表整個祖靈祭儀式圓滿完成 鄒族 簡介        鄒族居住在臺灣中部中高海拔的阿里山山區，祭典有豐收謝神的小米收穫祭典（homeyaya）與彰顯彪炳戰功的凱旋祭（mayasvi）。

Source 2:
作數月至數年，婚後男性跟隨妻子居住。民國 50 年代（1960 年代）之後，阿美族人與其他族群的接觸與通婚越來越頻繁，傳統從妻居的婚姻方式漸漸調整為嫁娶婚姻，家業也由男子代替女性承接，成為偏父系的社會。  領袖制度        阿美族最高領袖是大頭目，透過選賢與能制度，由地方領袖、男子年齡階級與司祭家代表在會議中選出。        近代阿美族的大頭目中以古拉斯 ‧ 馬亨亨最為有名。古拉斯 ‧ 馬亨亨（Kolas Mahengheng）是臺東阿美馬蘭社人，出生於清咸豐 2 年（1852），因為身材高大、聲音宏亮而被暱稱為「馬亨亨」。古拉斯 ‧ 馬亨亨（Kolas Mahengheng）在清末日初時期，經常往來於花東縱谷及東海岸各個部落，調解不少各族之間的紛爭與多起部落反抗事件，例如日本殖民統治初期的「七腳川事件」、「麻荖漏事件」等。

Source 3:
民國 89 年（2000）年臺東縣政府為肯定「馬亨亨」的成就，特別將通往臺東車站的外環道命名為「馬亨亨大道」。

Source 4:
民國50 年代（1960 年代）之後，阿美族人與其他族群的接觸與通婚越來越頻繁，傳統從妻居的婚姻方式漸漸調整為嫁娶婚